## School Ranking EDA

In [6]:
# 3rd Party Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
import geopandas as gpd
import folium
from shapely.geometry import Point

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

In [7]:
# import schools with coords

current_dir = Path.cwd()
parent_dir = current_dir.parent
gps_school = pd.read_csv(parent_dir / "2_data_cleaning" / "cleaned_csv" / "Combined_schools_with_coords.csv")
gps_school = gps_school.dropna(subset=["lat", "long"])

In [8]:
# School EDA
geometry = [Point(xy) for xy in zip(gps_school['long'], gps_school['lat'])]
gdf = gpd.GeoDataFrame(gps_school, geometry=geometry)

gdf.set_crs("EPSG:4326", allow_override=True, inplace=True)

m = folium.Map(location=[43.7, -79.4], zoom_start=11)

def get_color(value, column):
    if column == "EQAO % at Levels 3/4":
        if value <= 20:
            return 'red'
        elif value <= 40:
            return 'orange'
        elif value <= 60:
            return 'yellow'
        elif value <= 80:
            return 'green'
        else:
            return 'blue'
    elif column == "OSSLT % Successful":
        if value <= 50:
            return 'red'
        elif value <= 70:
            return 'orange'
        elif value <= 85:
            return 'yellow'
        elif value <= 95:
            return 'green'
        else:
            return 'blue'
    elif column == "Fraiser Score":
        if value <= 3:
            return 'red'
        elif value <= 5:
            return 'orange'
        elif value <= 7:
            return 'yellow'
        elif value <= 8:
            return 'green'
        else:
            return 'blue'
    return 'gray'

def add_circle_markers(map_obj, gdf, column, layer_name):
    layer = folium.FeatureGroup(name=layer_name)
    for idx, row in gdf.iterrows():
        value = row[column]
        if pd.notna(value):
            color = get_color(value, column)
            folium.CircleMarker(
                location=[row['lat'], row['long']],
                radius=8,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.6,
                popup=f"{row['School Name']}<br>{column}: {value}",
                tooltip=f"{row['School Name']}<br>{column}: {value}"
            ).add_to(layer)
    layer.add_to(map_obj)

add_circle_markers(m, gdf, "EQAO % at Levels 3/4", "EQAO % at Levels 3/4")
add_circle_markers(m, gdf, "OSSLT % Successful", "OSSLT % Successful")
add_circle_markers(m, gdf, "Fraiser Score", "Fraiser Score")

folium.LayerControl().add_to(m)

def add_legend(map_obj, column):
    if column == "EQAO % at Levels 3/4":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 50px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>EQAO % at Levels 3/4 Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-20<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 21-40<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 41-60<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 61-80<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 81-100<br>
            </div>
        """
    elif column == "OSSLT % Successful":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 220px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>OSSLT % Successful Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-50<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 51-70<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 71-85<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 86-95<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 96-100<br>
            </div>
        """
    elif column == "Fraiser Score":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 390px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>Fraiser Score Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-3<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 4-5<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 6-7<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 8<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 9-10<br>
            </div>
        """
    
    map_obj.get_root().html.add_child(folium.Element(legend))

add_legend(m, "EQAO % at Levels 3/4")
add_legend(m, "OSSLT % Successful")
add_legend(m, "Fraiser Score")

m
